In [1]:
pip install pandas nltk scikit-learn joblib pymssql


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import joblib
import pymssql
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

try:
    # Define connection parameters
    server = xx.xxx.xx.xxx'
    database = 'PineData'
    username = 'dbadmin'
    password = 'xxxxxxxxx'
    
    # Connect to the SQL server
    conn = pymssql.connect(server, username, password, database)
    cursor = conn.cursor()
    
    # Fetch Id and PostContent from the database
    sql = "SELECT Id, PostContent FROM [dbo].[NewsArticles]"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    # Convert SQL results to DataFrame
    data_from_sql = pd.DataFrame(rows, columns=['Id', 'PostContent'])
    
    # Preprocessing function
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # Preprocessing function with null check
    def preprocess(text):
        if text is None:
            return ''
        text = text.lower()  # Lowercase
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
        tokens = text.split()  # Tokenization
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stop words and lemmatize
        return ' '.join(tokens)
    
    # Apply preprocessing
    data_from_sql['PreprocessPostContent'] = data_from_sql['PostContent'].apply(preprocess)
    
    # Load the trained model and vectorizer
    model = joblib.load('newscategoriesprediction_model.pkl')
    vectorizer = joblib.load('tfidf_vectorizer.pkl')
    
    # Transform the preprocessed text data using the loaded vectorizer
    X_sql_tfidf = vectorizer.transform(data_from_sql['PreprocessPostContent'])
    
    # Sentiment analysis using NLTK VADER
    analyzer = SentimentIntensityAnalyzer()
    
    def sentiment_analysis(text):
        sentiment_scores = analyzer.polarity_scores(text)
        return sentiment_scores
    
    # Apply sentiment analysis and expand into separate columns
    data_from_sql['sentiment'] = data_from_sql['PreprocessPostContent'].apply(sentiment_analysis)
    sentiment_features_sql = pd.json_normalize(data_from_sql['sentiment'])
    
    # Combine TF-IDF features with sentiment features for the SQL data
    X_sql_combined = np.hstack((X_sql_tfidf.toarray(), sentiment_features_sql[['neg', 'neu', 'pos', 'compound']].values))
    
    # Drop the 'sentiment' and 'PreprocessPostContent' columns from the DataFrame
    data_from_sql.drop(columns=['sentiment', 'PreprocessPostContent'], inplace=True)
    
    # Predict labels using the loaded model
    predicted_labels = model.predict(X_sql_combined)
    
    # Add predictions to the DataFrame
    data_from_sql['Category'] = predicted_labels
    
    # Display the DataFrame with predictions (optional)
    print(data_from_sql)
    
    try:
        # Update Category column in SQL database based on Id
        for index, row in data_from_sql.iterrows():
            sql_update = '''
            UPDATE [dbo].[NewsArticles]
            SET Category = %s
            WHERE Id = %s
            '''
            params = (row['Category'], row['Id'])
            cursor.execute(sql_update, params)
        
        # Commit the transaction
        conn.commit()
        
        print("Category updated successfully in SQL database.")
    
    except Exception as e:
        print(f"Error updating category: {str(e)}")
        conn.rollback()

finally:
    # Close cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()


    Id                                        PostContent             Category
0    1  The Financial Intelligence Unit has slapped a ...            TERRORISM
1    6  Ottawa: Canada on Wednesday (local time) liste...            TERRORISM
2    7  The Gulf region has become an increasingly imp...            TERRORISM
3    9  KARACHI: Pakistan has called on the United Nat...            TERRORISM
4   10  As India flexes its newly acquired muscles for...            TERRORISM
5    2  The Financial Intelligence Unit (FIU) of India...            TERRORISM
6    5  India's Consulate General in Toronto reaffirms...            TERRORISM
7   11  The Union government is considering introducin...                 NDPS
8    3  New Delhi, June 20: Leading blockchain and cry...          CYBER CRIME
9    4  Financial Intelligence Unit penalised the comp...          CYBER CRIME
10   8  Ottawa on Wednesday listed Iran's Revolutionar...            TERRORISM
11  12  MUMBAI: Nawal Bajaj, a 1995-batch IPS office